Как строить нейронные сети?

Нейронные сети позволяют автоматизировать подбор признаковых пространств объектов (необязательно с линейными зависимостями).

Для этого как раз и используются функции активации между (линейными) слоями. Потому что без этой функции активации у нас бы линейный слой шел бы за линейным слоем. А линейная комбинация линейных слоев это все тот же **один** линейный слой. Добавив **нелинейность** мы все дальше можем уйти от линейного представления данных в более сложные объекты.
При этом руками нужно подбирать лишь функции активации.

Таким образом, мы получаем нейронную сеть, которая может описывать разные объекты с нелинейными зависимостями, все слои из которой можно дифференцировать (лучше бы, для метода обратного распространения ошибки и поиска градиента).

## Градиентный спуск и метод обратного распространения ошибки

**Нелинейные преобразования:**
* $ f(a) = \dfrac{1}{1 + e^a} $ -- sigmoid, сигмоида
* $ f(a) =  \tanh (a) $ -- гиперболические тангенс
* $ f(a) = \max (0, a) $ -- ReLU. Недифференцируемая. Но если рассматривать, что слева $f'(+0) = 0$, справа $f'(-0) = 1$. То есть производную посчитать можно. Функция не обязана быть *гладкой*.
* $ f(a) = \log(1 + e^a) $ -- softplus

### Строим нейронные сети. Базовые понятия.
**Слой** -- структурная единица нейронки, которая включает в себя линейное преобразование + нелинейная функция активации, которая применяется к результату линейного слоя.

Входной слой -- исходное представление данных.
Выходной слой -- итоговое представление данных: метка класса, вероятность класса, предсказанное число.

Все обучается с помощью **backpropagation, метода обратного распространения ошибки** -- методу обучения сложных функций.

Производная сложной функции:
$$ \dfrac{\delta L}{\delta x} = \dfrac{\delta L}{\delta z} \cdot \dfrac{\delta z}{\delta x}$$

В многомерном случае:
![Метод распространения ошибок от $f(x, y$](pics/nn_gradients.png)

![](pics/div_complicated_function.png)

### Backpropagation
Градиент -- направление наискорейшего *возрастания* функции.

Обучение моделей машинного обучения мы строим на минимизации функции ошибок. И для минимизации этой *loss-функции* мы используем метод градиентного спуска, то есть двигаемся в сторону антиградиента.

Нашу нейронную сеть можно представить как очень сложную функцию. Потому находить градиент нейронки мы также можем находить как производную сложной функции -- "дифференцировать" *шаг за шагом*.

Пример с картинки:

$$ \dfrac{\delta L}{\delta x} = \dfrac{\delta L}{\delta z} \cdot  \left(\dfrac{\delta L}{\delta x} + \dfrac{\delta L}{\delta y}\right)$$

Все эти свистопляски можно представить как *граф вычислений*:
![](pics/example-gradient.png)

Полносвязная нейронная сеть

- относительно хорошо справляется с классификацией
- по факту куча линейных слоев

![](pics/mlp_classification.png)
![](pics/mlp_penultimate_layer.png)

На последнем слое, чтоб превратить эти меры (не от 0 до 1) в вероятности принадлежности к классу применяется softmax-преобразование.

Необходимо, что сумма вероятностей принадлежности к определенному классу давала 1.

Преобразование softmax:
$$
    softmax(y_i) = \dfrac{e^{y_i}}{\sum e^{y_i}} \in  [0, 1], \\
    \sum softmax(y_i) = 1
$$

### Обучение MLP

Функция потерь для последнего слоя:
$$ logloss(y, p) = \log p_y(x) \rightarrow \max $$
или в общем виде функции потерь
$$ -logloss(y, p) = -\log p_y(x) \rightarrow \min $$

Итоговая задача оптимизации:
$$ \dfrac{1}{l} \sum logloss(y, p(x, \theta)) \rightarrow \min_\theta $$

Стохастический градиентный спуск
* Выбираем батч из обучающей выборки
* Обновляем веса backpro на этом батче, а не на всей выборке



![](pics/neuron_layer.png)

## Функции активации

### Сигмоида
$$ f(a) = \dfrac{1}{1 + e^a} $$
+ отображение в [0, 1]
+ интерпретируется как вероятность
+ дифференцируема
- обладает "хвостами", в которых производная почти 0
- тяжелая частная производная, тяжело считать
- затухание градиента
- нет центрирования 
* используется в *реккурентных* нейронных сетях

### Гиперболический тангенс
$$ f(a) = \tanh(a) $$
+ отображение в [-1, 1]
+ центрированно
- затухание градиента на хвостах
- все еще надо считать экспоненту
* используется в *реккурентных* нейронных сетях

### ReLU, 
+ отображение в [0, $+\inf$]
+ намного легче высчитывать и самую функцию
+ производную намного легче считать: либо 0, либо 1
- не центрированно в нуле
- слева на хвосте затухают градиенты

### Leaky ReLU
$$f(x) = \max(0.01x, x)$$
+ не затухает
+ легко вычисляется
+ простая производная


### Parametric PReLU
$$f(x) = \max(\alpha x, x)$$
* просто почему бы и нет
* параметр альфа можно настраивает с помощью backprop

### ELU
$$
    f(x) = \begin{cases}
        x, \quad x > 0 \\
        \alpha (\exp(x) - 1), \quad x \leq 0
    \end{cases}
$$
+ все плюсы ReLU
+ не затухает
- нужно считать экспоненту

![](pics/activation_functions.png)

Функция активации выбирается в соответствии от типа нейронных сетей: реккурентные, сверточные и так далее.

![](pics/advices_activation.png)

## Fancy neural networks